In [4]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.2 MB/s eta 0:00:00


In [3]:
import pandas as pd

# Load the dataset
file_path = '/content/preprocessed_phishing_emails_dataset.csv'
dataset = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(dataset.head())

# Split the dataset into training, validation, and test sets
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(dataset, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)


                                          Email Text  Email Type
0  re      disc  uniformitarianism  re    sex  la...         0.0
1  the other side of  galicismos   galicismo  is ...         0.0
2  re  equistar deal tickets are you still availa...         0.0
3  hello i am your hot lil horny toy\n    i am th...         1.0
4  software at incredibly low prices    lower   d...         1.0


In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [10]:
# Print the column names of the DataFrame
print(dataset.columns)
# Check for NaN values in the 'Email Text' column
print("Number of NaN values in 'Email Text' column:", train_df['Email Text'].isna().sum())

# Drop rows with NaN values or fill them with a placeholder string
train_df = train_df.dropna(subset=['Email Text'])
# or
train_df['Email Text'] = train_df['Email Text'].fillna('placeholder')
# Convert the elements in the 'Email Text' column to strings
train_df['Email Text'] = train_df['Email Text'].astype(str)

Index(['Email Text', 'Email Type'], dtype='object')
Number of NaN values in 'Email Text' column: 17


<ipython-input-10-8f7c69e228c1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Email Text'] = train_df['Email Text'].fillna('placeholder')
<ipython-input-10-8f7c69e228c1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Email Text'] = train_df['Email Text'].astype(str)


In [9]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader, TensorDataset

# Load the BERT tokenizer for bert-large-uncased
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased' )

# Tokenize the email texts in the training, validation, and test sets
train_inputs = tokenizer(train_df['Email Text'].tolist(), padding=True, truncation=True, return_tensors='pt', max_length=256)
val_inputs = tokenizer(val_df['Email Text'].tolist(), padding=True, truncation=True, return_tensors='pt', max_length=256)
test_inputs = tokenizer(test_df['Email Text'].tolist(), padding=True, truncation=True, return_tensors='pt', max_length=256)
# Create DataLoaders for the training, validation, and test sets
train_dataloader = DataLoader(TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], torch.tensor(train_df['Email Type'].values)), batch_size=8, shuffle=True)
val_dataloader = DataLoader(TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], torch.tensor(val_df['Email Type'].values)), batch_size=8)
test_dataloader = DataLoader(TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], torch.tensor(test_df['Email Type'].values)), batch_size=8)



ValueError: ignored

In [ ]:
#For URL
# Map the string labels to numeric values
label_mapping = {
    'good': 0,
    'bad': 1
}

# Apply the mapping to the 'Label' column in the DataFrames
train_df['Label'] = train_df['Label'].map(label_mapping)
val_df['Label'] = val_df['Label'].map(label_mapping)
test_df['Label'] = test_df['Label'].map(label_mapping)

# Check for any NaN values in case there are unrecognized labels
print("NaN values in train_df:", train_df['Label'].isna().sum())
print("NaN values in val_df:", val_df['Label'].isna().sum())
print("NaN values in test_df:", test_df['Label'].isna().sum())

# Optionally, handle NaN values (if any) by dropping rows or assigning a default value


NaN values in train_df: 0
NaN values in val_df: 0
NaN values in test_df: 0


In [ ]:
# Define the path to save the model
def save_to_drive():
  model_save_path = "/content/drive/MyDrive/my_trained_model_url"

  # Save the model's state dictionary and configuration
  model.save_pretrained(model_save_path)
  tokenizer.save_pretrained(model_save_path)



In [ ]:
from transformers import BertForSequenceClassification

# Load the pre-trained bert-large-uncased model
model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#for email
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
num_epochs = 3

# Define the optimizer, loss function, and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * num_epochs)

# Fine-tune the model on the training set
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Validate the model's performance on the validation set
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"Epoch {epoch + 1}, Validation Loss: {val_loss / total}, Validation Accuracy: {correct /total}")

save_to_drive()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader

# Training parameters
num_epochs = 3
batch_size = 8
gradient_accumulation_steps = 4
learning_rate = 2e-5

# Optimizer, Loss Function, Scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * num_epochs // gradient_accumulation_steps)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training Loop with Gradient Accumulation and Early Stopping
best_val_loss = float('inf')
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, batch in enumerate(train_dataloader):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()
        running_loss += loss.item()

        if (i + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

    print(f"Epoch {epoch + 1}, Training Loss: {running_loss / len(train_dataloader)}")

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_dataloader)
    val_accuracy = correct / total
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

    # Early Stopping and Model Checkpointing
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print("Saved best model!")

# Save final model
torch.save(model.state_dict(), "final_model.pth")


NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#from google.colab import files

# Download the model ZIP file to your local machine
#files.download('my_trained_model_email.zip')
save_to_drive()

In [ ]:
!zip -r my_trained_model_email.zip my_trained_model_email


  adding: my_trained_model_email/ (stored 0%)
  adding: my_trained_model_email/vocab.txt (deflated 53%)
  adding: my_trained_model_email/pytorch_model.bin (deflated 7%)
  adding: my_trained_model_email/config.json (deflated 48%)
  adding: my_trained_model_email/special_tokens_map.json (deflated 42%)
  adding: my_trained_model_email/tokenizer_config.json (deflated 45%)
